In [3]:
#!/usr/bin/env python3
"""
Simplified: Extract 10 stratified episodes PER TASK (independent).
Each task independently stratifies into 10 tiers and selects 1 episode per tier.
Total: 60 episodes (10 per task × 6 tasks)
"""

import json
import shutil
from pathlib import Path
import numpy as np

# Configuration
SOURCE_TASK_DIRS = [
    "/storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/dreamer_button-press-topdown",
    "/storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/dreamer_door-open",
    "/storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/dreamer_drawer-close",
    "/storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/dreamer_peg-insert-side",
    "/storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/dreamer_pick-place",
    "/storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/dreamer_push"
]

OUTPUT_FOLDER = "/storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/stratified_per_task_10"

N_TIERS_PER_TASK = 10  # Each task split into 10 performance tiers
EPISODES_PER_TIER = 1  # 1 episode per tier → 10 total per task

def main():
    print(f"Per-Task Stratified Extraction")
    print(f"Configuration: Each task gets {N_TIERS_PER_TASK} episodes\n")
    
    output_path = Path(OUTPUT_FOLDER)
    output_path.mkdir(parents=True, exist_ok=True)
    
    total_copied = 0
    
    for task_dir in SOURCE_TASK_DIRS:
        task_name = Path(task_dir).name
        task_short = "_".join(task_name.split("_")[1:])  # Remove 'dreamer_' prefix
        
        metrics_file = Path(task_dir) / "metrics.jsonl"
        train_eps_folder = Path(task_dir) / "train_eps"
        
        if not metrics_file.exists() or not train_eps_folder.exists():
            print(f"⚠ {task_short}: Missing files, skipping")
            continue
        
        # Load episodes for this task
        episodes = []
        with open(metrics_file, 'r') as f:
            for line in f:
                data = json.loads(line)
                if 'train_episodes' in data and 'train_return' in data:
                    ep_idx = int(data['train_episodes']) - 1
                    train_return = data['train_return']
                    episodes.append((ep_idx, train_return))
        
        # Sort by score (descending)
        sorted_episodes = sorted(episodes, key=lambda x: x[1], reverse=True)
        
        # Stratify this task independently
        tier_size = len(sorted_episodes) // N_TIERS_PER_TASK
        selected_episodes = []
        
        print(f"{task_short}:")
        print(f"  Total episodes: {len(episodes)}")
        print(f"  Extracting 1 episode from each of {N_TIERS_PER_TASK} tiers")
        
        for tier_idx in range(N_TIERS_PER_TASK):
            tier_start = tier_idx * tier_size
            tier_end = (tier_idx + 1) * tier_size if tier_idx < N_TIERS_PER_TASK - 1 else len(sorted_episodes)
            
            tier_episodes = sorted_episodes[tier_start:tier_end]
            
            if len(tier_episodes) > 0:
                # Pick middle episode from this tier
                mid_idx = len(tier_episodes) // 2
                ep_idx = tier_episodes[mid_idx][0]
                selected_episodes.append(ep_idx)
        
        # Copy selected episodes
        task_output_folder = output_path / task_short
        task_output_folder.mkdir(parents=True, exist_ok=True)
        
        all_npz_files = sorted(train_eps_folder.glob('*.npz'))
        copied = 0
        
        for ep_idx in sorted(selected_episodes):
            if ep_idx < len(all_npz_files):
                src = all_npz_files[ep_idx]
                dst = task_output_folder / src.name
                shutil.copy2(src, dst)
                copied += 1
        
        total_copied += copied
        print(f"  ✓ Copied {copied} episodes\n")
    
    print(f"TOTAL: {total_copied} episodes across all tasks")
    print(f"Output: {output_path}")

if __name__ == "__main__":
    main()

Per-Task Stratified Extraction
Configuration: Each task gets 10 episodes

button-press-topdown:
  Total episodes: 1740
  Extracting 1 episode from each of 10 tiers
  ✓ Copied 10 episodes

door-open:
  Total episodes: 1740
  Extracting 1 episode from each of 10 tiers
  ✓ Copied 10 episodes

drawer-close:
  Total episodes: 1740
  Extracting 1 episode from each of 10 tiers
  ✓ Copied 10 episodes

peg-insert-side:
  Total episodes: 1740
  Extracting 1 episode from each of 10 tiers
  ✓ Copied 10 episodes

pick-place:
  Total episodes: 1740
  Extracting 1 episode from each of 10 tiers
  ✓ Copied 10 episodes

push:
  Total episodes: 1740
  Extracting 1 episode from each of 10 tiers
  ✓ Copied 10 episodes

TOTAL: 60 episodes across all tasks
Output: /storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/stratified_per_task_10


In [1]:
#!/usr/bin/env python3
"""
Modified: Extract 10 stratified episodes PER TASK (independent),
but only from the TOP 50% of episodes for that task.

Each task:
1. Sort episodes by train_return (descending).
2. Keep only the top 50% highest-return episodes.
3. Stratify that subset into 10 tiers and select 1 episode per tier.

Total: 60 episodes (10 per task × 6 tasks), all from the top half.
"""

import json
import shutil
from pathlib import Path
import numpy as np

# Configuration
SOURCE_TASK_DIRS = [
    "/storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/dreamer_button-press-topdown",
    "/storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/dreamer_door-open",
    "/storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/dreamer_drawer-close",
    "/storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/dreamer_peg-insert-side",
    "/storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/dreamer_pick-place",
    "/storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/dreamer_push"
]

OUTPUT_FOLDER = "/storage/ssd1/richtsai1103/vid2act/pretrain/metaworld/pretrain_data/stratified_per_task_10_top50"

N_TIERS_PER_TASK = 10  # Each task split into 10 performance tiers (within top 50%)
EPISODES_PER_TIER = 1  # 1 episode per tier → 10 total per task


def main():
    print(f"Per-Task Stratified Extraction (Top 50%)")
    print(f"Configuration: Each task gets {N_TIERS_PER_TASK} episodes from its top 50% of returns\n")
    
    output_path = Path(OUTPUT_FOLDER)
    output_path.mkdir(parents=True, exist_ok=True)
    
    total_copied = 0
    
    for task_dir in SOURCE_TASK_DIRS:
        task_name = Path(task_dir).name
        task_short = "_".join(task_name.split("_")[1:])  # Remove 'dreamer_' prefix
        
        metrics_file = Path(task_dir) / "metrics.jsonl"
        train_eps_folder = Path(task_dir) / "train_eps"
        
        if not metrics_file.exists() or not train_eps_folder.exists():
            print(f"⚠ {task_short}: Missing files, skipping")
            continue
        
        # Load episodes for this task
        episodes = []
        with open(metrics_file, 'r') as f:
            for line in f:
                data = json.loads(line)
                if 'train_episodes' in data and 'train_return' in data:
                    ep_idx = int(data['train_episodes']) - 1
                    train_return = data['train_return']
                    episodes.append((ep_idx, train_return))
        
        if len(episodes) == 0:
            print(f"⚠ {task_short}: No episodes found, skipping")
            continue

        # Sort by score (descending)
        sorted_episodes = sorted(episodes, key=lambda x: x[1], reverse=True)

        # --- NEW: keep only top 50% of episodes ---
        half_len = int(np.ceil(len(sorted_episodes) * 0.5))
        top_episodes = sorted_episodes[:half_len]
        # -------------------------------------------
        
        # Stratify this task independently (only using the top 50%)
        tier_size = max(1, len(top_episodes) // N_TIERS_PER_TASK)
        selected_episodes = []
        
        print(f"{task_short}:")
        print(f"  Total episodes: {len(episodes)}")
        print(f"  Top 50% episodes: {len(top_episodes)}")
        print(f"  Extracting {EPISODES_PER_TIER} episode from each of {N_TIERS_PER_TASK} tiers (within top 50%)")
        
        for tier_idx in range(N_TIERS_PER_TASK):
            tier_start = tier_idx * tier_size
            # Make sure the last tier goes to the end
            if tier_idx < N_TIERS_PER_TASK - 1:
                tier_end = (tier_idx + 1) * tier_size
            else:
                tier_end = len(top_episodes)
            
            tier_episodes = top_episodes[tier_start:tier_end]
            
            if len(tier_episodes) > 0:
                # Pick middle episode from this tier
                mid_idx = len(tier_episodes) // 2
                ep_idx = tier_episodes[mid_idx][0]
                selected_episodes.append(ep_idx)
        
        # Deduplicate in case of any overlap due to small dataset
        selected_episodes = sorted(list(set(selected_episodes)))[:N_TIERS_PER_TASK]
        
        # Copy selected episodes
        task_output_folder = output_path / task_short
        task_output_folder.mkdir(parents=True, exist_ok=True)
        
        all_npz_files = sorted(train_eps_folder.glob('*.npz'))
        copied = 0
        
        for ep_idx in sorted(selected_episodes):
            if ep_idx < len(all_npz_files):
                src = all_npz_files[ep_idx]
                dst = task_output_folder / src.name
                shutil.copy2(src, dst)
                copied += 1
        
        total_copied += copied
        print(f"  ✓ Copied {copied} episodes\n")
    
    print(f"TOTAL: {total_copied} episodes across all tasks")
    print(f"Output: {output_path}")


if __name__ == "__main__":
    main()


Per-Task Stratified Extraction (Top 50%)
Configuration: Each task gets 10 episodes from its top 50% of returns

button-press-topdown:
  Total episodes: 1740
  Top 50% episodes: 870
  Extracting 1 episode from each of 10 tiers (within top 50%)
  ✓ Copied 10 episodes

door-open:
  Total episodes: 1740
  Top 50% episodes: 870
  Extracting 1 episode from each of 10 tiers (within top 50%)
  ✓ Copied 10 episodes

drawer-close:
  Total episodes: 1740
  Top 50% episodes: 870
  Extracting 1 episode from each of 10 tiers (within top 50%)
  ✓ Copied 10 episodes

peg-insert-side:
  Total episodes: 1740
  Top 50% episodes: 870
  Extracting 1 episode from each of 10 tiers (within top 50%)
  ✓ Copied 10 episodes

pick-place:
  Total episodes: 1740
  Top 50% episodes: 870
  Extracting 1 episode from each of 10 tiers (within top 50%)
  ✓ Copied 10 episodes

push:
  Total episodes: 1740
  Top 50% episodes: 870
  Extracting 1 episode from each of 10 tiers (within top 50%)
  ✓ Copied 10 episodes

TOTAL: 6

In [1]:
import json
import os
from pathlib import Path
from math import isfinite

from torch.utils.tensorboard import SummaryWriter


def load_metrics_from_jsonl(jsonl_path):
    metrics = []
    with open(jsonl_path, "r") as f:
        for line in f:
            line = line.strip()
            if not line:
                continue
            try:
                metric = json.loads(line)
                metrics.append(metric)
            except json.JSONDecodeError as e:
                print(f"Warning: Could not parse line: {line[:100]}... Error: {e}")
    return metrics


def is_valid_scalar(value):
    if value is None or isinstance(value, (bool, str)):
        return False
    try:
        v = float(value)
        return isfinite(v)
    except (TypeError, ValueError):
        return False


def write_to_tensorboard(metrics, logdir="./runs/metrics"):
    Path(logdir).mkdir(parents=True, exist_ok=True)
    writer = SummaryWriter(log_dir=logdir)

    total_scalars = 0

    for metric_dict in metrics:
        if "step" not in metric_dict:
            continue

        step = int(metric_dict["step"])

        for key, value in metric_dict.items():
            if key == "step":
                continue
            if not is_valid_scalar(value):
                continue

            value = float(value)
            # group under 'scalars/' tag as you wanted
            writer.add_scalar(f"scalars/{key}", value, global_step=step)
            total_scalars += 1

    writer.flush()
    writer.close()

    print(f"Wrote {total_scalars} scalar points to TensorBoard at {logdir}")


In [2]:
metrics_file = "/storage/ssd1/richtsai1103/vid2act/log/metaworld/mt6/drawer_open/original_seed0/metrics.jsonl"
metrics = load_metrics_from_jsonl(metrics_file)
write_to_tensorboard(metrics, logdir="./runs/metrics")


Wrote 3828 scalar points to TensorBoard at ./runs/metrics
